In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import sys
import random

from tqdm import tqdm

import re
import string

import os

import shutil

import json

In [ ]:
f_train = '/content/drive/MyDrive/data/simplified-nq-train.jsonl'
f_test = '/content/drive/MyDrive/data/simplified-nq-test.jsonl'
num_train_samples = 307372
num_test_samples = 346

In [ ]:
df_val_id = pd.read_csv('/content/drive/MyDrive/data/fine_data/val_id.csv', dtype=str)
df_val_id.head()

,example_id
0,-4415223548557328381
1,-706775569507012323
2,-8247431609310444236
3,178097630653295544
4,7206764020223190386


In [ ]:
set_val_id = set(df_val_id['example_id'].values.tolist())
print(len(set_val_id))

3075


In [ ]:
def getValSamples(set_id, filename=f_train):
    val_sam = []
    with open(filename) as f:
        progress = tqdm(f, total=num_train_samples)
        for sam_count, line in enumerate(progress): 
            if sam_count >= num_train_samples:
                break   
            data = json.loads(line)
            
            example_id = str(data['example_id'])
            if example_id in set_id: # process this sample
                val_sam.append(data)

    val_sam_df = pd.DataFrame(val_sam)
    val_sam_df.to_json('/content/drive/MyDrive/data/simplified-nq-val.json')

In [ ]:
getValSamples(set_val_id)

100%|██████████| 307372/307372 [05:35<00:00, 915.42it/s] 


In [ ]:
def getValFile(set_id, filename=f_train, debug=False): 
    num = 0  
    lines = [] 
    with open(filename) as f:
        progress = tqdm(f, total=num_train_samples)
        for sam_count, line in enumerate(progress): 
            if sam_count >= num_train_samples:
                break   
            data = json.loads(line)
            
            example_id = str(data['example_id'])
            if example_id in set_id: # process this sample
                num += 1 
                ans_id = data['annotations'][0]['long_answer']['candidate_index']
                
                long_id = str(example_id) + '_long'
                short_id = str(example_id) + '_short'
                long_string = ''
                short_string = ''

                if ans_id > -1:
                    lan_start = data['long_answer_candidates'][ans_id]['start_token']
                    lan_stop = data['long_answer_candidates'][ans_id]['end_token']

                    yes_no = data['annotations'][0]['yes_no_answer']
                    if yes_no == "NONE":
                        is_yes_no = False
                    else:
                        is_yes_no = True

                    list_sans = data['annotations'][0]['short_answers']

                    san_start = lan_stop
                    san_stop = lan_start
                    is_san = False

                    for san in list_sans:
                        this_start = san['start_token']
                        this_stop = san['end_token']
                        if san_start > this_start: # start token should be smallest among short answers' start 
                            san_start = this_start
                        if san_stop < this_stop: # stop token should be largest among short answers' stop
                            san_stop = this_stop
                    
                    if san_start < san_stop:
                        is_san = True
                    
                    if is_san:
                        short_string = str(san_start) + ":" + str(san_stop)

                    if is_yes_no:
                        short_string = yes_no
                    
                    long_string = str(lan_start) + ':' + str(lan_stop)
                
                long_line = {}
                long_line['example_id'] = long_id
                long_line['PredictionString'] = long_string

                # lines.append(long_line)

                short_line = {}
                short_line['example_id'] = short_id
                short_line['PredictionString'] = short_string

                lines.append(short_line)

                if debug:
                    if num%100 == 0:
                        document_text = data['document_text']
                        document_split = document_text.split()

                        if long_string != '':
                            lanTokens = long_string.split(':')
                            lanStart = int(lanTokens[0])
                            lanStop = int(lanTokens[1])
                            print(example_id, "LONG ANSWER: ", ' '.join(document_split[lanStart:lanStop]))
                            
                            if short_string != '':
                                if short_string == 'NO' or short_string == 'YES':
                                    print(example_id, "SHORT ANSWER: ", short_string)
                                else:
                                    sanTokens = short_string.split(':')
                                    sanStart = int(sanTokens[0])
                                    sanStop = int(sanTokens[1])
                                    print(example_id, "SHORT ANSWER: ", ' '.join(document_split[sanStart:sanStop]))
                        else:
                            print(example_id, "NO ANSWER")
    print("Number processed samples = ", num)
    return lines

In [ ]:
lines = getValFile(set_val_id, debug=True)

  3%|▎         | 8087/307372 [00:06<05:06, 977.09it/s] 

7800541314044975804 NO ANSWER


  6%|▌         | 18893/307372 [00:14<03:38, 1320.73it/s]

3159184035169356505 NO ANSWER


 10%|█         | 31635/307372 [00:22<03:50, 1197.50it/s]

...

-1534897927286362423 LONG ANSWER:  <P> The most successful breed in the modern era since Best in Show was introduced has been the English Cocker Spaniel . Of the breed 's seven show titles , all but one of them were owned and bred by Herbert Summers Lloyd ( known predominantly as H.S. Lloyd ) from the `` of Ware '' kennel . Only four dogs have won Best in Show on more than one occasion , and on three of these occasions they were English Cocker Spaniels owned by Lloyd . The fourth occasion was a Labrador Retriever named Bramshaw Bob , owned by Countess Lorna Howe who is the second most successful breeder in the show 's history as she also won Best in Show once more with another Labrador in 1937 , Ch . Cheveralla Ben of Banchory . Although no dog has won Crufts more than once since H.S. Lloyd 's Tracey Witch of Ware in 1950 , owner Jackie Lorimer won the title in 1993 with Irish Setter Sh Ch . Danaway Debonair and again with the dog 's son , Sh Ch . Caspians Intrepid in 1999 . </P>
-1534

 98%|█████████▊| 300824/307372 [04:34<00:02, 2286.82it/s]

3170191793954002798 LONG ANSWER:  <P> McQueen , Mcqueen , and MacQueen , Macqueen are English - language surnames derived from Scottish Gaelic . There have been several differing etymologies given for the surnames ; as well as several differing ways to represent the surname in modern Scottish Gaelic . The surnames are not among the most common surnames in the United Kingdom , Australia , nor the United States . </P>


100%|██████████| 307372/307372 [04:38<00:00, 1102.51it/s]


Number processed samples =  3075


In [ ]:
lines_df = pd.DataFrame(lines)

In [ ]:
lines_df.head()

,example_id,PredictionString
0,4158175306918787233_short,408:410
1,-1957133654292017851_short,
2,-1273245364552286191_short,
3,-8663891343543535834_short,3126:3128
4,8161832609608306276_short,


In [ ]:
sorted_df = lines_df.sort_values('example_id')

In [ ]:
sorted_df.head()

,example_id,PredictionString
877,-1000683513392992283_short,
488,-1006288523617376129_short,
1899,-1008973731705801750_short,
210,-1009784761503721527_short,1203:1204
3052,-1012377094725254051_short,


In [ ]:
sorted_df.shape[0]

3075

In [ ]:
sorted_df.to_csv('/content/drive/MyDrive/data/fine_data/validation_short.csv', index=False, columns=['example_id', 'PredictionString'])

In [ ]:
def getValCandidates(set_id, filename=f_train, debug=False): 
    num = 0  
    list_docs = [] 
    with open(filename) as f:
        progress = tqdm(f, total=num_train_samples)
        for sam_count, line in enumerate(progress): 
            if sam_count >= num_train_samples:
                break  

            data = json.loads(line)

            example_id = str(data['example_id'])
            if example_id in set_id: # process this sample
                list_candidates = data['long_answer_candidates']
                one_doc = {'example_id': example_id, "list_candidates": list_candidates}
                list_docs.append(one_doc)
                num += 1
            
    print("Number processed samples = ", num)
    return list_docs

In [ ]:
list_doc_candidates = getValCandidates(set_val_id)

100%|██████████| 307372/307372 [04:30<00:00, 1135.42it/s]

Number processed samples =  3075


In [ ]:
list_doc_candidates_df = pd.DataFrame(list_doc_candidates)

In [ ]:
list_doc_candidates_df.head()

,example_id,list_candidates
0,4158175306918787233,"[{'start_token': 8, 'top_level': True, 'end_to..."
1,-1957133654292017851,"[{'start_token': 26, 'top_level': True, 'end_t..."
2,-1273245364552286191,"[{'start_token': 14, 'top_level': True, 'end_t..."
3,-8663891343543535834,"[{'start_token': 66, 'top_level': True, 'end_t..."
4,8161832609608306276,"[{'start_token': 8, 'top_level': True, 'end_to..."


In [ ]:
list_doc_candidates_df.to_pickle('/content/drive/MyDrive/data/fine_data/val_candidates.csv')

In [ ]:
test_df = pd.read_pickle('/content/drive/MyDrive/data/fine_data/val_candidates.csv')

In [ ]:
test_df.head()

,example_id,list_candidates
0,4158175306918787233,"[{'start_token': 8, 'top_level': True, 'end_to..."
1,-1957133654292017851,"[{'start_token': 26, 'top_level': True, 'end_t..."
2,-1273245364552286191,"[{'start_token': 14, 'top_level': True, 'end_t..."
3,-8663891343543535834,"[{'start_token': 66, 'top_level': True, 'end_t..."
4,8161832609608306276,"[{'start_token': 8, 'top_level': True, 'end_to..."


In [ ]:
for rowid, row in test_df.iterrows():
    if (rowid == 0):
        print(row)
        print(row['list_candidates'])
        cands = row['list_candidates']
        for cand in cands:
            print(cand['start_token'])

In [ ]:
test_df.shape[0]

3075